# Data Crawling

In [1]:
# 환경 구축
!pip install selenium
!pip install BeautifulSoup4
!apt update
!apt install chromium-chromedriver

     |████████████████████████████████| 911kB 2.7MB/s 
Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic I

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import WebDriverException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import UnexpectedAlertPresentException

from bs4 import BeautifulSoup
import time
import csv
import datetime

In [50]:
del df

In [54]:
import pandas as pd
df = pd.DataFrame(columns=['name', 'content', 'date'])
df = df.append({'name': '가나다라', 'content': 'text', 'date': '2020-01-01'}, ignore_index=True)
df

,name,content,date
0,가나다라,text,2020-01-01


In [60]:
def health_chosun_crawling():
    #공공보건 포털사이트의 뉴스 기사를 크롤링함
    site = 'http://health.chosun.com/list.html?menu=&more_menu=&more_smenu=&nowcode=1&type=&pn=1'

    #csv 파일 저장을 위한 f, utf-8을 인코딩으로 사용
    #f = open(path, 'w',encoding='utf-8')

    # 데이터 프레임 생성
    df = pd.DataFrame(columns=['name', 'date', 'content'])
    #파일 객체를 생성
    wr = csv.writer(f)

    row = 1

    break_all = False
    #chrome으로 driver를 실행
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)

    #조건 성립 시간을 위한 wait 객체 설정
    wait = WebDriverWait(driver,20)

    #driver의 브라우저 크기를 최대화
    driver.maximize_window()
    #최대화를 하지 않았을 경우 마우스 포인터가 정확히 div를 클릭하지 못하여 크롤링을 못할 수 있음

    #웹의 컴포넌트 로딩을 기다리기 위한 implicitly_wait 함수, 로딩이 끝나면 바로 진행된다.
    driver.implicitly_wait(3)

    #미리 설정해둔 공공보건 포털사이트로 이동한다.
    driver.get(site)

    #크롤링할 기사의 날짜 범위 설정
    print("크롤링할 기사의 날짜 범위를 설정하세요")

    date_entry = input('시작 날짜(YYYY-MM-DD 형식) : ')
    year, month, day = map(int, date_entry.split('-'))
    date1 = datetime.date(year, month, day)

    date_entry = input('종료 날짜(YYYY-MM-DD 형식) : ')
    year, month, day = map(int, date_entry.split('-'))
    date2 = datetime.date(year, month, day)

    #기사의 제목, 내용, 생성날짜를 저장할 변수를 설정한다.
    name=""
    content=""
    date=""

    while True:
        time.sleep(1)
        idx = 1
        list_num = len(driver.find_elements_by_css_selector("dl.list_item"))
        while True:
            #변수들의 초기화
            name=""
            content=""
            date=""
            while True:
                news_list = driver.find_element_by_css_selector("div#list_area dl:nth-child(" + str(idx) +")")
                news = news_list.find_element_by_css_selector('dt a')
                news_text = news_list.text
                #잡다한 만화, 책광고 등등의 기사를 제외시킴
                if news.text.find("카툰") != -1 or news.text.find("해랑")!= -1 or news.text.find("서적") != -1 or news.text.find("튼튼선생") != -1 or news.text.find("양냥") != -1 or news_text.find("비만클리닉") != -1:
                    print(str(row) + "오류발생")
                    idx += 1
                    if(list_num < idx):
                        break;
                else:
                    break;      
            if(list_num < idx):
                break;
            news.click()
            try:
                alert = driver.switch_to.alert
                alert.accept()
                time.sleep(2)
            except :
                pass
            try :
                time.sleep(1)
                name = driver.find_element_by_css_selector("h1#news_title_text_id").text
                date = driver.find_element_by_css_selector("p#date_text").text.split(" ")[1]
                parts = driver.find_elements_by_css_selector("div.par p")

                year, month, day = map(int, date.split('.'))
                date3 = datetime.date(year, month, day)

                for part in parts:
                    if part.text.find("기자") == -1 and part.text.replace(" ","") != "":
                        content += part.text
                    else:
                        break
                # 해당 기사의 날짜가 사용자 지정 범위 내인 경우에만 csv파일에 작성
                # 시작 날짜보다 이전에 작성된 기사라면 크롤링 종료
                if date1 > date3:
                    print("finish")
                    return df

                if date1 <= date3 and date3 <= date2:
                    #wr.writerow([row, name, date, content])
                    # append
                    df = df.append({'name': name, 'date': date, 'content': content}, ignore_index=True)
                    print(str(row) + " " + name + " / " + str(date3))

                row +=1
                driver.back()
                driver.implicitly_wait(5)
                idx += 1
                if(list_num < idx):
                    break;            
            except (AttributeError, NoSuchElementException, UnexpectedAlertPresentException):
                driver.back()
                idx += 1
                if(list_num < idx):
                    break;
                 
        page_now = driver.find_element_by_css_selector("div.paginate ul.paginate_num span.listAct").text.split(" ")[0]
        print("page " + str(int(page_now)) + " complete")
        url = "http://health.chosun.com/list.html?menu=&more_menu=&more_smenu=&nowcode=1&type=&pn=" + str(int(page_now) + 1)
        driver.get(url)

In [61]:
path = "health_chosun.csv"
result = health_chosun_crawling()
result

크롤링할 기사의 날짜 범위를 설정하세요
시작 날짜(YYYY-MM-DD 형식) : 2020-11-11
종료 날짜(YYYY-MM-DD 형식) : 2020-11-11
1 손 시리면 수족냉증? 저리고 아프면 ‘이것’ 의심 / 2020-11-11
2 갱년기 여성 비만하면 ‘치밀 유방’ 감소하지만… / 2020-11-11
3 만병의 근원 '염증' 줄이는 식품 5 / 2020-11-11
4 씨젠, 매출 1조 가나?… 불붙는 진단키트 수출 / 2020-11-11
5 '심장' 지키는 법… 밀가루, 가공육 그리고 '이것' 피하라 / 2020-11-11
6 고려대 안암병원 정형외과 박종웅 교수, 대한수부외과학회 회장 취임 / 2020-11-11
7 [질병백과 TV] 벌어진 다리 그냥 두면… '내측관절염' 악화될 수도 / 2020-11-11
8 안지영 활동중단… 불안 증세 완화하려면? / 2020-11-11
9 대한결핵및호흡기학회, 12일부터 국제컨퍼런스 개최 / 2020-11-11
10 '향기'로 여성 성기능을 개선한다? / 2020-11-11
11 아침에 '이것' 한 잔… 체중감량, 콜레스테롤 배출에 도움 / 2020-11-11
12 [질병백과 TV] 어느 날 갑자기 '청력 저하'… 1/3은 청력 완전히 잃는다 / 2020-11-11
13 수두·볼거리·백일해 절반으로 '뚝'… “코로나 생활방역 효과” / 2020-11-11
14 목소리만으로 후두암 판별 / 2020-11-11
15 심장병 예방에 도움 주는 '견과류'는? / 2020-11-11
16 당신이 '뱃살을 빼야 하는' 의외의 이유 / 2020-11-11
17 매복된 어금니… '바이오급속교정'으로 문체 치아만 견인한다 / 2020-11-11
18 불면증, 한번 겪기 시작하면 낫기 어렵다 / 2020-11-11
19 아직도 한 손으로 양치질하세요? '양손 양치질'의 비밀… / 2020-11-11
20 동아에스티, 中항서제약 면역항암제 국내 독점 판권 확보 / 2020-11-11
21 정부 "코로나 백신, 내년 하반기 접종 

,name,date,content
0,손 시리면 수족냉증? 저리고 아프면 ‘이것’ 의심,2020.11.11,겨울은 평소 손발이 차가운 사람들에게 유독 힘든 계절이다. 기온이 급격히 떨어지면서...
1,갱년기 여성 비만하면 ‘치밀 유방’ 감소하지만…,2020.11.11,갱년기 여성이 비만하면 치밀 유방이 감소한다는 연구가 나왔다.치밀 유방이란 유방에 ...
2,만병의 근원 '염증' 줄이는 식품 5,2020.11.11,염증은 만병의 근원이다. 심지어 암(癌)조차 체내 염증에 의해 발생한다는 설이 있다...
3,"씨젠, 매출 1조 가나?… 불붙는 진단키트 수출",2020.11.11,"국내 기업의 대규모 코로나19 진단키트 수출이 이어지고 있다. 씨젠, 랩지노믹스 등..."
4,"'심장' 지키는 법… 밀가루, 가공육 그리고 '이것' 피하라",2020.11.11,기온이 떨어지면 몸의 혈관이 급격히 수축해 심근경색 등 심장질환 발생 위험이 커진다...
5,"고려대 안암병원 정형외과 박종웅 교수, 대한수부외과학회 회장 취임",2020.11.11,고려대 안암병원 정형외과 박종웅 교수가 2020년 11월 6 일 부로 제 28대 대...
6,[질병백과 TV] 벌어진 다리 그냥 두면… '내측관절염' 악화될 수도,2020.11.11,'오(O)다리'는 발목 안쪽을 붙인 상태로 똑바로 섰을 때 다리가 반듯하지 않고 무...
7,안지영 활동중단… 불안 증세 완화하려면?,2020.11.11,가수 볼빨간사춘기 안지영(26)이 지속적인 불안 증세를 겪어 활동을 중단한다.오늘(...
8,"대한결핵및호흡기학회, 12일부터 국제컨퍼런스 개최",2020.11.11,대한결핵 및 호흡기학회가 11월 12일(수)-13일(목) 양일간 대한결핵 및 호흡기...
9,'향기'로 여성 성기능을 개선한다?,2020.11.11,향기를 약으로 사용하는 ‘아로마테라피(Aromatherapy)’. 아로마테라피는 허...


In [38]:
ls

sample_data/
